# CNN

In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
def load_and_preprocess_images(folder_path, label):
    images = []
    labels = []
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        img = Image.open(image_path)
        img = img.resize((64, 64))  # Resize to match the input shape of the model
        img = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
        labels.append(label)  # Assign label to images in this folder
    return np.array(images), np.array(labels)

In [3]:
# Define the paths to your two folders
folder1_path = r'C:\Users\User\Downloads\Dataset\zebra long wing'
folder2_path = r'C:\Users\User\Downloads\Dataset\wood satyr'
0ad_and_preprocess_images(folder1_path, label=0)
images_folder2, labels_folder2 = load_and_preprocess_images(folder2_path, label=1)

# Concatenate the data from both folders
train_images = np.concatenate((images_folder1, images_folder2), axis=0)
train_labels = np.concatenate((labels_folder1, labels_folder2), axis=0)

In [4]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu')
])

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification, so 1 neuron with sigmoid activation


In [5]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary classification loss
              metrics=['accuracy'])

In [6]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
5/5 [==============================] - 5s 416ms/step - loss: 0.6493 - accuracy: 0.5132 - val_loss: 0.8349 - val_accuracy: 0.0000e+00
Epoch 2/10
5/5 [==============================] - 1s 250ms/step - loss: 0.4483 - accuracy: 0.6974 - val_loss: 0.6412 - val_accuracy: 1.0000
Epoch 3/10
5/5 [==============================] - 1s 256ms/step - loss: 0.3210 - accuracy: 0.9211 - val_loss: 0.6202 - val_accuracy: 0.9211
Epoch 4/10
5/5 [==============================] - 1s 256ms/step - loss: 0.1997 - accuracy: 0.9803 - val_loss: 0.3099 - val_accuracy: 0.9737
Epoch 5/10
5/5 [==============================] - 1s 260ms/step - loss: 0.0702 - accuracy: 1.0000 - val_loss: 0.0535 - val_accuracy: 0.9737
Epoch 6/10
5/5 [==============================] - 1s 286ms/step - loss: 0.0142 - accuracy: 0.9934 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 1s 293ms/step - loss: 0.1186 - accuracy: 0.9671 - val_loss: 6.8048e-04 - val_accuracy: 1.0000
Epoch 8/10
5

In [7]:
val_loss, val_accuracy = model.evaluate(train_images, train_labels, verbose=0)

print(f'Validation accuracy: {val_accuracy}')

Validation accuracy: 0.9947368502616882


# VG16

In [8]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [9]:
data_dir = r"C:\Users\User\Downloads\Dataset"
classes = os.listdir(data_dir)
X = []
y = []
for class_name in classes:
    class_path = os.path.join(data_dir, class_name)
if class_name == "wood satyr":
    label = 0
else:
    label = 1
for img_name in os.listdir(class_path):
    img_path = os.path.join(class_path, img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # VGG16 expects RGB format
    img = cv2.resize(img, (224, 224))
    X.append(img)
    y.append(label)
X = np.array(X)

# y = to_categorical(y) # one-hot encode the labels
X_train, X_test, y_train, y_test = train_test_split(X, np.array(y), test_size=0.2, random_state=42, stratify=y)

# Load VGG16 model without the top (fully connected) layers
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the VGG16 layers so they're not trainable
for layer in vgg.layers:
    layer.trainable = False
    
# Create a new model on top of the VGG16 convolutional base
model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid')) # 2 classes

58889256/58889256 [==============================] - 557s 9us/step


In [10]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Preprocess the input data for VGG16
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/10


C:\Users\User\anaconda3\Lib\site-packages\tensorflow\python\util\dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


3/3 [==============================] - 51s 16s/step - loss: 0.0000e+00 - accuracy: 0.7692 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 2/10
3/3 [==============================] - 49s 16s/step - loss: 0.0000e+00 - accuracy: 0.6538 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 3/10
3/3 [==============================] - 46s 15s/step - loss: 0.0000e+00 - accuracy: 0.7436 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 4/10
3/3 [==============================] - 47s 16s/step - loss: 0.0000e+00 - accuracy: 0.6923 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 5/10
3/3 [==============================] - 49s 17s/step - loss: 0.0000e+00 - accuracy: 0.6538 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 6/10
3/3 [==============================] - 47s 16s/step - loss: 0.0000e+00 - accuracy: 0.6154 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 7/10
3/3 [==============================] - 39s 13s/step - loss: 0.0000e+00 - accuracy: 0.6667 - val_loss: 0.0000e+00 - val